In [70]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import math
import unidecode

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinatti',
                        'pirates': 'Pittsburg',
                        'dodgers': 'Los Angeles Dodgers',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinatti': 'NL',
          'Pittsburg': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']

In [159]:
class Batter:
    """Class that holds information for a Batter.
    
    Attributes:
        team: team name of player
        name: player name
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks):
        """Initializes values for this class"""
        
        self.team = team
        self.name = name
        self.singles = singles
        self.doubles = singles + doubles
        self.triples = singles + doubles + triples
        self.home_runs = singles + doubles + triples + home_runs
        self.walks = singles + doubles + triples + home_runs + walks
        self.cum_singles = 0
        self.cum_doubles = 0
        self.cum_triples = 0
        self.cum_home_runs = 0
        self.cum_walks = 0
        self.cum_pa = 0
        self.cum_runs = 0
        self.cum_rbi = 0
                
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if(self.cum_pa-self.cum_walks>0):
            return(float((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs)/(self.cum_pa-self.cum_walks)))
        else:
            return 0    
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa>0):
            return((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs+self.cum_walks)/(self.cum_pa))
        else:
            return 0
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form"""
        if(self.cum_pa-self.cum_walks>0):
            return((self.cum_singles+2*self.cum_doubles+3*self.cum_triples+4*self.cum_home_runs)/(self.cum_pa-self.cum_walks))
        else:
            return 0
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg() + self.obp())
    def print_stats(self):
        """Prints some stats of a player"""

        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str(self.cum_home_runs))
        
class Pitcher(Batter):
    """Class that holds information for a pitcher.
    
    Attributes:
        team: team name of player
        name: player name
        position: player position
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks):
        """Initializes values for this class"""
        
        Batter.__init__(self, team, name, singles, doubles, triples, home_runs, walks)
        
class Team:
    """Class that holds information for a team.
    
    Attributes:
        city: team city
        roster: batting lineup of team in list form
        batting_index: position in batting order of current batter
        rotation_index: position in rotation of current pitcher
        runners: list representing whether a runner is on each base
        runs: number of runs fr a team in a game
        wins: number of wins for a team
        losses: number of losses for a team
        cumulative runs: total runs for a team over a span of games
    """
    def __init__(self, city, league, batter_df, pitcher_df, starters):
        """Initializes values for this class"""
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.pitcher_df = pitcher_df
        self.starters = starters
        self.pitching_staff = pitcher_df
        self.batting_index = 0
        self.rotation_index = 0
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        self.pitchers = []
        self.batters = []
       # print(batter_df)
       # print(pitcher_df)
       # print(starters)
        
            
        for index, batter in batter_df.iterrows():
            num_singles = batter['H']-batter['2B']-batter['3B']-batter['HR']
            pa = batter['AB'] + batter['BB']
            singles = round(num_singles/pa, 3)
            doubles = round(batter['2B']/pa, 3)
            triples = round(batter['3B']/pa, 3)
            home_runs = round(batter['HR']/pa, 3)
            walks = round(batter['BB']/pa, 3)

            self.batters.append(Batter(self.city, batter['PLAYER'], singles, doubles, triples, home_runs, walks))

        for index, pitcher in pitcher_df.iterrows():
            ip = math.modf(pitcher['IP'])
            outs = int(ip[0]*10 + ip[1]*3)
            total_batters = outs + pitcher['BB'] + pitcher['H']
            hit_ratio = pitcher['H']/total_batters
            singles = round(hit_ratio*.64, 3)
            doubles = round(hit_ratio*.2, 3)
            triples = round(hit_ratio*.02, 3)
            home_runs = round(hit_ratio*.14, 3)
            walks = round(pitcher['BB']/total_batters, 3)

            self.pitchers.append(Pitcher(self.city, pitcher['PLAYER'], singles, doubles, triples, home_runs, walks))

        def make_lineup(batters, starters):
            lineup = []
            
            for starter in starters['lineup']:
                for batter in batters:
                    if(batter.name==starter):
                        lineup.append(batter)
                        print(starter)
                    
            return lineup
        #def choose_pitcher(pitchers, starters):
        
        self.lineup = make_lineup(self.batters, self.starters)
        #self.pitcher = choose_pitcher(self.pitchers, self.starters)
        
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started"""
        self.batting_index = 0
        self.runners = [None, None, None, 0]
        self.runs = 0
    def reset_runners(self):
        """Resets baserunners after each half inning"""
        self.runners = [None, None, None, 0]
        
class Game:
    """Class that holds information for a Game.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team
        
    def get_result(self, batter, pitcher, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """
        batter.cum_pa+=1
        pitcher.cum_pa+=1
        
        if(num<batter.singles):
            batter.cum_singles+=1
            pitcher.cum_singles+=1
            return "single"
        elif(num<batter.doubles):
            batter.cum_doubles+=1
            pitcher.cum_doubles+=1
            return "double"
        elif(num<batter.triples):
            batter.cum_triples+=1
            pitcher.cum_triples+=1
            return "triple"
        elif(num<batter.home_runs):
            batter.cum_home_runs+=1
            pitcher.cum_home_runs+=1
            return "home_run"
        elif(num<batter.walks):
            batter.cum_walks+=1
            pitcher.cum_walks+=1
            return "walk"
        elif(num<1):
            return "out"
        elif(num<pitcher.singles+1):
            batter.cum_singles+=1
            pitcher.cum_singles+=1
            return "single"
        elif(num<pitcher.doubles+1):
            batter.cum_doubles+=1
            pitcher.cum_doubles+=1
            return "double"
        elif(num<pitcher.triples+1):
            batter.cum_triples+=1
            pitcher.cum_triples+=1
            return "triple"
        elif(num<pitcher.home_runs+1):
            batter.cum_home_runs+=1
            pitcher.cum_home_runs+=1
            return "home_run"
        elif(num<pitcher.walks+1):
            batter.cum_walks+=1
            pitcher.cum_walks+=1
            return "walk"
        else:
            return "out"
        
    def move_runners(self, team, pitcher, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            pitcher: pitcher pitching to team
            result: str of result for batter (ie "walk")
        """
        if(result=="single"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.lineup[team.batting_index]

        elif(result=="double"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.lineup[team.batting_index]
            team.runners[0] = None

        elif(result=="triple"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.lineup[team.batting_index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="home_run"):
            team.lineup[team.batting_index].cum_runs+=1
            pitcher.cum_runs+=1
            team.lineup[team.batting_index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="walk"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.lineup[team.batting_index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        temp_2.cum_runs+=1
                        pitcher.cum_runs+=1
                        team.lineup[team.batting_index].cum_rbi+=1
                        team.runners[3]+=1

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0  
            
    def print_result(self):
        """Printed score of a game""" 
        print("final score: " + self.away_team.name + " " + str(self.away_team.runs) + " " + self.home_team.name + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams""" 
        print(self.away_team.name + ": " + str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(self.home_team.name + ": " + str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played""" 
        self.away_team.restart()
        self.home_team.restart() 
        inning = 1
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                outs = 0
                
                if(side==self.away_team):
                    pitcher = self.home_team.pitcher
                elif(side==self.home_team):
                    pitcher = self.away_team.pitcher

                while(outs<3):
                    num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.batting_index], pitcher, num)
                    if(result=="out"):
                        outs+=1
                    else:
                        self.move_runners(side, pitcher, result)
                        
                    side.batting_index+=1
                    side.batting_index = side.batting_index%8
                    
                side.reset_runners()
            inning+=1
        
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team
        
    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)
            g.play() 
            
    def reset_season(self, teams):
        
        for team in teams:
            team.__init__(team.name, team.batters, team.pitchers, team.starters)
                        

In [3]:
def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)

In [26]:
def batter_stats():

    url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
    links = [url+str(i) for i in range(1, 350, 50)]
    headers = {'User-Agent': header_name}

    batter_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            batter_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        batter_stats = pd.concat([batter_stats, df], ignore_index=True)

    return batter_stats

In [27]:
def pitcher_stats():
    url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
    links = [url+str(i) for i in range(1, 500, 50)]
    headers = {'User-Agent': header_name}

    pitcher_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            pitcher_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

    return pitcher_stats

In [28]:
def column_type(batter_stats, pitcher_stats):
    string_columns = ['Pos', 'Name', 'PLAYER']
    int_columns = ['ID', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
                   'RBI', 'SB', 'CS', 'BB', 'SO', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB']
    float_columns = ['BA', 'OBP', 'SLG', 'OPS']

    for column in batter_stats.columns:
        if(column in int_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_float(row, column), axis=1)

    string_columns = ['Pos', 'Name', 'PLAYER']
    int_columns = ['ID', 'Age', 'W', 'L', 'G', 'GS', 'GF', 'CG', 'SHO', 'SV', 'H', 'R',
                   'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+']
    float_columns = ['W-L%', 'ERA', 'IP', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9',
                    'SO/W']


    for column in pitcher_stats.columns:
        if(column in int_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_float(row, column), axis=1)     
            
    return batter_stats, pitcher_stats

In [64]:
def projected_starters():

    url = 'https://www.mlb.com/news/projecting-every-mlb-lineup-rotation'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    starters = {}

    for body in soup.findAll('div', attrs={'class': 'article-item__body'}):
        pitchers = []
        batters = []
        closer = None

        for team in body.findAll('p'):
            if(team.strong and team.strong.get_text().lower() in nickname_to_city_dict.keys()):
                current_team = team.strong.get_text().lower()

                for batter in team.findAll(['forge-entity', 'a']):
                    batters.append(unidecode.unidecode(batter.get_text()))
            elif(team.strong and team.strong.get_text()=='Pitchers'):
                for pitcher in team.findAll(['forge-entity', 'a']):
                    pitchers.append(unidecode.unidecode(pitcher.get_text()))
            elif(team.strong and team.strong.get_text()=='Closer:'):
                closer = unidecode.unidecode(team.find(['forge-entity', 'a']).get_text())
            else:
                continue

            if(closer!=None):
                starters.update({nickname_to_city_dict[current_team]: {'lineup': batters, 'rotation': pitchers, 'closer': closer}})
                pitchers = []
                batters = []
                closer = None
                
    return starters

In [156]:
def create_teams(batter_stats, pitcher_stats, starters):
    url_dict = {}
    url_start = 'https://www.espn.com/mlb/team/roster/_/name/'
    team_dict = {}

    for acronym, city in acronym_to_city_dict.items():
        url_dict.update({url_start+acronym.lower(): city})

    for url, city in url_dict.items():
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        player_id_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for section in soup.findAll('section'):
            for player in section.findAll('td', attrs={'class': 'Table__TD'}):
                row = player.find('a', attrs={'class': 'AnchorLink'})
                pos = player.find('span')

                if(row and row.get_text()!=''):
                    if(section.div.get_text()=='Pitchers'):
                        player_id_dict.update({row.get_text(): {int(row['href'].split('/')[-1]): 'P'}})
                    else:
                        player_id_dict.update({row.get_text(): {int(row['href'].split('/')[-1]): 'H'}})
                   
        for name, df in player_id_dict.items():
            for ID, pos in df.items():
                if(pos=='P'):
                    if(ID in pitcher_stats['ID'].tolist()):
                        stats = pitcher_stats.loc[pitcher_stats['ID']== ID]
                        col_headers = stats.columns
                    else:
                        stats = pd.Series({'ID': ID, 'PLAYER': name, 'YRS': 1, 'G': 30, 'GS': 0, 'SH': 0, 'IP': 30, 'H': 30,
                                           'ER': 15, 'BB': 12, 'SO': 30, 'W': 2, 'L': 3, 'SV': 0, 'ERA': 4.50, 'CG': 0})
                        col_headers = stats.index
                    if(pitcher_df.empty):
                        pitcher_df = pd.DataFrame(columns=col_headers)
                    
                    pitcher_df = pitcher_df.append(stats, ignore_index=True)

                elif(pos=='H'):
                    if(ID in batter_stats['ID'].tolist()):
                        stats = batter_stats.loc[batter_stats['ID']== ID]
                        col_headers = stats.columns
                    else:
                        stats = pd.Series({'ID': ID, 'PLAYER': name, 'YRS': 1, 'G': 60, 'AB': 200, 'R': 25, 'H': 50, '2B': 10,
                                           '3B': 1, 'HR': 5, 'RBI': 25, 'BB': 20, 'SO': 50, 'SB': 5, 'CS': 1, 'BA': .250})
                        col_headers = stats.index
                        
                    if(batter_df.empty):
                        batter_df = pd.DataFrame(columns=col_headers)

                    batter_df = batter_df.append(stats, ignore_index=True)

        team_dict.update({city: Team(city, league[city], batter_df, pitcher_df, starters[city])})
        
    return team_dict

In [66]:
batter_stats_df = batter_stats()
pitcher_stats_df = pitcher_stats()
batter_stats_df, pitcher_stats_df = column_type(batter_stats_df, pitcher_stats_df)
starters_dict = projected_starters()


In [160]:
team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict)

Ronald Acuna Jr.
Ozzie Albies
Freddie Freeman
Marcell Ozuna
Travis d'Arnaud
Dansby Swanson
Johan Camargo
Austin Riley
Ender Inciarte
Trea Turner
Adam Eaton
Juan Soto
Carter Kieboom
Asdrubal Cabrera
Starlin Castro
Howie Kendrick
Eric Thames
Kurt Suzuki
Victor Robles
Jeff McNeil
Pete Alonso
Michael Conforto
J.D. Davis
Robinson Cano
Wilson Ramos
Brandon Nimmo
Amed Rosario
Andrew McCutchen
J.T. Realmuto
Bryce Harper
Rhys Hoskins
Didi Gregorius
Jean Segura
Scott Kingery
Adam Haseley
Jonathan Villar
Brian Anderson
Corey Dickerson
Jesus Aguilar
Jorge Alfaro
Isan Diaz
Miguel Rojas
Lewis Brinson


KeyboardInterrupt: 

In [72]:
g = Game(team_dict['Atlanta'], team_dict['Kansas City'])
g.play()
g.print_result()

KeyError: 'Atlanta'

In [73]:
s.reset_season([team_dict['Atlanta'], team_dict['Kansas City']])
s = Season(team_dict['Atlanta'], team_dict['Kansas City'])
s.simulate_season(162)
s.print_record()

KeyError: 'Atlanta'

In [5]:
def split_stats():   
    url = 'http://www.espn.com/mlb/player/splits/_/id/31283/type/batting3'

    headers = {'User-Agent': header_name}

    platoon_dict = {}
    data_columns = ['Split', 'Year', 'Tm', 'team_ID', 'Pos', 'POS', 'pos_season', 'Name', 'player', 'Age', 'age', 'G', 'PA', 'AB', 
                    'R', 'H', '2B', '3B', 'HR', 'SB', 'CS', 'BB', 'SO', 'HBP']



    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    left_columns = ['Name', 'Bats']
    right_columns = []
    name = soup.h1.get_text()

    for bio in soup.findAll('div', attrs={'class': 'player-bio'}):
        for info in bio.findAll('ul', attrs={'class': 'general-info'}):
            bats = info.get_text().split('Bats')[1][2]

    for table in soup.findAll('table', attrs={'class': 'tablehead'}):
        labels = table.find('tr', attrs={'class': 'colhead'})

        for header_value in labels.findAll('td'):
            left_columns.append(header_value.get_text()+'L')
            right_columns.append(header_value.get_text()+'R')

        df = pd.DataFrame(columns=(left_columns+right_columns))            
        player_values = [name, bats]

        for column_value in table.findAll('tr', attrs={'class': ['oddrow', 'evenrow']}):
            if(column_value.td.get_text() in ['vs. Left', 'vs. Right']):
                for value in column_value.findAll('td'):
                    player_values.append(value.get_text())

        df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

    df.drop(['OverallL', 'OverallR'], axis=1, inplace=True)   
    print(df)

               Name Bats  ABL  RL   HL 2BL 3BL HRL RBIL BBL  ... RBIR  BBR  \
0  Christian Yelich    L  503  82  148  30   3  25   75  53  ...  213  175   

  HBPR  SOR SBR CSR  AVGR  OBPR  SLGR  OPSR  
0   12  282  51   5  .318  .410  .582  .992  

[1 rows x 34 columns]


In [45]:
for index, batter in batter_df.iterrows():
    print(batter.name)

NameError: name 'batter_df' is not defined

In [69]:
batter_stats_df.loc[batter_stats_df['PLAYER']=='Ender Inciarte']

,ID,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA


In [77]:
player_id_dict

NameError: name 'player_id_dict' is not defined